# HW3

In [ ]:
import numpy as np
import seaborn as sns
sns.set_context('talk')
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'

## Problem 1: Linear Regression Class [40pts]

### Part 1: Base Class

In [ ]:
class Regression(object):
    
    def __init__(self):
        self.params = {}
    
    def get_params(self):
        # return best fit params (B-hat) from fit()
        return self.params
    
    def set_params(self, **kwargs):
        # TODO
        pass
    
    def fit(self, X, y):
        # fits linear model supplied from subclass to X and y 
        # and stores best fit coeffs (b_hat) in params dict
        """Fit model."""
    
    def predict(self, X):
        # TODO
        pass 
    
    def score(self, X, y):
        # TODO
        pass

### Part 2: OLS Linear Regression

In [ ]:
class LinearRegression(Regression):
    
    def fit(self, X, y):
        #super.fit()
        # create design matrix (padding with ones column)
        intercept_coeffs = np.ones(len(X))
        X = np.column_stack((intercept_coeffs, X))
        
        # compute B_hat
        X_inv = np.linalg.pinv(X)
        B_hat = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)
        
        self.params['intercept'] = B_hat[0]
        self.params['coeffs'] = B_hat[1:]

In [ ]:
# fake data
x1 = np.array([3.1, 4.3, 5.4, 6.1, 6.9, 8.1])
x2 = np.array([2.9, 4.1, 5.2, 6.3, 7.4, 8.3])
X = np.column_stack((x1, x2))
y = np.array([5, 8.4, 10.1, 11.7, 14.5, 16])

lin_fit = LinearRegression()

lin_fit.fit(X, y)
lin_fit.get_params()

## Problem 2 [10pts]

### Part 1: Create a module

###### Included in directory as `MathCS207.py`

### Part 2: Import a whole module and use it

In [1]:
import MathCS207

b = 2.

# show output from module
print(
    f"input: {b}\n"
    f"addition: {MathCS207.add(b, b)}\n"
    f"subtraction: {MathCS207.subtract(b, b)}\n"
    f"multiplication: {MathCS207.multiply(b, b)}\n"
    f"division: {MathCS207.divide(b, b)}"
)

input: 2.0
addition: 4.0
subtraction: 0.0
multiplication: 4.0
division: 1.0


### Part 3: Import a single function from a module and use it

In [ ]:
from MathCS207 import add

print(add(b, b))

### Part 4: Import a module by creating an alias of it and then use the alias

In [ ]:
import MathCS207 as mathcs

print(mathcs.add(b, b))

### Part 5: List every function definition inside the module MathCS207

In [2]:
print(dir(MathCS207))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'add', 'divide', 'multiply', 'subtract']
